## Importing the CIFAR-100 dataset

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.preprocessing import LabelBinarizer

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data(label_mode='fine')

print(x_train.shape, y_train.shape)

(50000, 32, 32, 3) (50000, 1)


In [3]:
print(tf.config.list_physical_devices('GPU'))

[]


In [4]:
x_train = (x_train / 255 - 0.5) * 2
x_test = (x_test / 255 - 0.5) * 2

enc = LabelBinarizer()

y_train = enc.fit_transform(y_train)
y_test = enc.fit_transform(y_test)


In [5]:
print(x_test[0].shape)

(32, 32, 3)


## Inception v4 Implementation

In [7]:
#import tensorflow as tf
from keras.layers import Dense, Conv2D, Input, MaxPooling2D, BatchNormalization, Activation, AveragePooling2D, Dropout, Concatenate, Flatten
#from keras.layers import *
from keras import Model


# TODO: Go to Inceptionv2 paper to understand Conv2D_BN and fine tune parameters potentially
def Conv2DBatchNorm(x, filters, kernel_size=(3, 3), strides=(1, 1), padding='same'):

    x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding)(x)
            #    kernel_regularizer=keras.regularizers.l2(0.00004),
            #    kernel_initializer=keras.initializers.VarianceScaling(scale=0.2, mode='fan_in', distribution='normal', seed=None)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    return x

def Stem(x):
    print("pre-stem size:", x.shape)
    x = Conv2DBatchNorm(x, 32, (3, 3), strides=(2, 2), padding='valid')
    x = Conv2DBatchNorm(x, 32, (3, 3), padding='valid')
    x = Conv2DBatchNorm(x, 64, (3, 3))

    branch_0 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)
    branch_1 = Conv2DBatchNorm(x, 96, (3, 3), strides=(2, 2), padding='valid')

    x = Concatenate()([branch_0, branch_1])

    branch_0 = Conv2DBatchNorm(x, 64, (1, 1))
    branch_0 = Conv2DBatchNorm(branch_0, 96, (3, 3), padding='valid')

    branch_1 = Conv2DBatchNorm(x, 64, (1, 1))
    branch_1 = Conv2DBatchNorm(branch_1, 64, (1, 7))
    branch_1 = Conv2DBatchNorm(branch_1, 64, (7, 1))
    branch_1 = Conv2DBatchNorm(branch_1, 96, (3, 3), padding='valid')

    x = Concatenate()([branch_0, branch_1])

    branch_0 = Conv2DBatchNorm(x, 192, (3, 3), strides=(2, 2), padding='valid')
    branch_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(x)

    x = Concatenate()([branch_0, branch_1])

    print("post-stem size:", x.shape)

    return x


def InceptionA(x):
    print("pre-A size:", x.shape)

    branch_0 = AveragePooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    branch_0 = Conv2DBatchNorm(branch_0, 96, (1, 1))

    branch_1 = Conv2DBatchNorm(x, 96, (1, 1))

    branch_2 = Conv2DBatchNorm(x, 64, (1, 1))
    branch_2 = Conv2DBatchNorm(branch_2, 96, (3, 3))

    branch_3 = Conv2DBatchNorm(x, 64, (1, 1))
    branch_3 = Conv2DBatchNorm(branch_3, 96, (3, 3))
    branch_3 = Conv2DBatchNorm(branch_3, 96, (3, 3))

    x = Concatenate()([branch_0, branch_1, branch_2, branch_3])
    print("post-A size:", x.shape)

    return x


def InceptionB(x):
    print("pre-B size:", x.shape)

    branch_0 = AveragePooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    branch_0 = Conv2DBatchNorm(branch_0, 128, (1, 1))

    branch_1 = Conv2DBatchNorm(x, 384, (1, 1))

    branch_2 = Conv2DBatchNorm(x, 192, (1, 1))
    branch_2 = Conv2DBatchNorm(branch_2, 224, (1, 7))
    branch_2 = Conv2DBatchNorm(branch_2, 256, (7, 1))

    branch_3 = Conv2DBatchNorm(x, 192, (1, 1))
    branch_3 = Conv2DBatchNorm(branch_3, 192, (1, 7))
    branch_3 = Conv2DBatchNorm(branch_3, 224, (7, 1))
    branch_3 = Conv2DBatchNorm(branch_3, 224, (1, 7))
    branch_3 = Conv2DBatchNorm(branch_3, 256, (7, 1))

    x = Concatenate()([branch_0, branch_1, branch_2, branch_3])
    print("post-B size:", x.shape)

    return x


def InceptionC(x):
    print("pre-C size:", x.shape)

    branch_0 = AveragePooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    branch_0 = Conv2DBatchNorm(branch_0, 256, (1, 1))

    branch_1 = Conv2DBatchNorm(x, 256, (1, 1))

    branch_2 = Conv2DBatchNorm(x, 384, (1, 1))
    branch_2_0 = Conv2DBatchNorm(branch_2, 256, (1, 3))
    branch_2_1 = Conv2DBatchNorm(branch_2, 256, (3, 1))

    branch_3 = Conv2DBatchNorm(x, 384, (1, 1))
    branch_3 = Conv2DBatchNorm(branch_3, 448, (1, 3))
    branch_3 = Conv2DBatchNorm(branch_3, 512, (3, 1))
    branch_3_0 = Conv2DBatchNorm(branch_3, 256, (3, 1))
    branch_3_1 = Conv2DBatchNorm(branch_3, 256, (1, 3))

    x = Concatenate()([branch_0, branch_1, branch_2_0, branch_2_1, branch_3_0, branch_3_1])
    print("post-C size:", x.shape)

    return x


def ReductionA(x):
    print("pre-RedA size:", x.shape)

    branch_0 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)

    branch_1 = Conv2DBatchNorm(x, 384, (3, 3), strides=(2, 2), padding='valid')

    branch_2 = Conv2DBatchNorm(x, 192, (1, 1))
    branch_2 = Conv2DBatchNorm(branch_2, 224, (3, 3))
    branch_2 = Conv2DBatchNorm(branch_2, 256, (3, 3), strides=(2, 2), padding='valid')

    x = Concatenate()([branch_0, branch_1, branch_2])
    print("post-RedA size:", x.shape)

    return x


def ReductionB(x):
    print("pre-RedB size:", x.shape)

    branch_0 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)

    branch_1 = Conv2DBatchNorm(x, 192, (1, 1))
    branch_1 = Conv2DBatchNorm(branch_1, 192, (3, 3), strides=(2, 2), padding='valid')

    branch_2 = Conv2DBatchNorm(x, 256, (1, 1))
    branch_2 = Conv2DBatchNorm(branch_2, 256, (1, 7))
    branch_2 = Conv2DBatchNorm(branch_2, 320, (7, 1))
    branch_2 = Conv2DBatchNorm(branch_2, 320, (3, 3), strides=(2, 2), padding='valid')

    x = Concatenate()([branch_0, branch_1, branch_2])
    print("post-RedB size:", x.shape)

    return x


def MyInception():
    inputs = Input(shape=(32, 32, 3), name='input')

    x = Stem(inputs)

    for _ in range(4):
        x = InceptionA(x)

    x = ReductionA(x)

    for _ in range(7):
        x = InceptionB(x)

    x = ReductionB(x)

    for _ in range(3):
        x = InceptionC(x)

    x = AveragePooling2D()(x)
    x = Dropout(0.2)(x)
    x = Flatten()(x)

    outputs = Dense(100, activation='softmax', name='output')(x)
    model = Model(inputs=inputs, outputs=outputs, name='MyInception')

    return model


model = MyInception()

model.compile()

model.summary()

pre-stem size: (None, 32, 32, 3)
post-stem size: (None, 1, 1, 384)
pre-A size: (None, 1, 1, 384)
post-A size: (None, 1, 1, 384)
pre-A size: (None, 1, 1, 384)
post-A size: (None, 1, 1, 384)
pre-B size: (None, 1, 1, 384)
post-B size: (None, 1, 1, 1024)
pre-B size: (None, 1, 1, 1024)
post-B size: (None, 1, 1, 1024)
pre-B size: (None, 1, 1, 1024)
post-B size: (None, 1, 1, 1024)
pre-B size: (None, 1, 1, 1024)
post-B size: (None, 1, 1, 1024)
pre-C size: (None, 1, 1, 1024)
post-C size: (None, 1, 1, 1536)


ValueError: Exception encountered when calling layer "average_pooling2d_22" (type AveragePooling2D).

Negative dimension size caused by subtracting 2 from 1 for '{{node average_pooling2d_22/AvgPool}} = AvgPool[T=DT_FLOAT, data_format="NHWC", ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1]](Placeholder)' with input shapes: [?,1,1,1536].

Call arguments received by layer "average_pooling2d_22" (type AveragePooling2D):
  • inputs=tf.Tensor(shape=(None, 1, 1, 1536), dtype=float32)